In [481]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as st

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import xgboost as xgb

import math


In [482]:
# Get to know the data
df_test = pd.read_csv("test.csv")
df_train = pd.read_csv("train.csv")
df_test["YearBuilt"]

0       1961
1       1958
2       1997
3       1998
4       1992
        ... 
1454    1970
1455    1970
1456    1960
1457    1992
1458    1993
Name: YearBuilt, Length: 1459, dtype: int64

In [483]:
quantitative_columns = ['PoolArea', 'ScreenPorch', '3SsnPorch', 'EnclosedPorch', 'OpenPorchSF', 'WoodDeckSF',
                        'GarageArea', 'GarageCars', 'MiscVal', 'Fireplaces', 'TotRmsAbvGrd', 'GrLivArea', 
                        'LowQualFinSF', '1stFlrSF', 'TotalBsmtSF', 'BsmtUnfSF', 'BsmtFinSF2',
                        'BsmtFinSF1', 'MasVnrArea', 'LotArea', 'LotFrontage', 'MSSubClass', 'YearBuilt', 'YearRemodAdd',
                        'GarageYrBlt', 'YrSold', '2ndFlrSF', 'HalfBath', 'FullBath', 'BsmtHalfBath',
                        'BsmtFullBath']

In [484]:
a = df_train.isna().mean().loc[df_train.isna().mean() != 0.0].index
b = df_test.isna().mean().loc[df_test.isna().mean() != 0.0].index
c = set(b) - set(a)
not_use_columns = list(c - set(quantitative_columns))
not_use_columns

['Exterior2nd',
 'Exterior1st',
 'SaleType',
 'Functional',
 'KitchenQual',
 'MSZoning',
 'Utilities']

In [485]:
def data_preprocessing(df_train, df_test):
    # delete not use columns
    df_test = df_test.drop(not_use_columns, axis=1)
    df_train = df_train.drop(not_use_columns, axis=1)
    
    #for quantitative data
    for c in quantitative_columns:
        df_train[c] = df_train[c].fillna(df_train[c].mean()).apply(lambda x: float(x))
        df_test[c] = df_test[c].fillna(df_train[c].mean()).apply(lambda x: float(x))

    #for categorical
    for c in set(df_train.columns) - set(quantitative_columns):
        df_train[c] = df_train[c].fillna("None").apply(lambda x: str(x))
        df_test[c] = df_test[c].fillna("None").apply(lambda x: str(x))
    return df_train, df_test

In [486]:
#transform the categorical data to label
"""
input : data frames (test and train)
output : data frame with label encoded columns
"""
def Label_encoding(df_train, df_test):
    for c in df_train.columns:
        if c not in quantitative_columns:
            print(c)
            le = LabelEncoder()
            le.fit(df_train[c])
            df_train[c] = le.transform(df_train[c])
            df_test[c] = le.transform(df_test[c])
    return df_train, df_test

In [487]:
def build_xgb_model(x_train, x_valid, y_train, y_valid):
    """
    Build Model function
    
    This function output is a Scikit ML Pipeline that process text messages
    according to NLP best-practice and apply a classifier.
    return a trained model and score
    """
 
    #change the datatype to fit to xgb
    dtrain = xgb.DMatrix(x_train, label = y_train)
    dvalid = xgb.DMatrix(x_valid, label = y_valid)
    
    #making model
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    num_round = 50
    
    
    #setting parameters
    parameters_tobe_selected = {
    'eta': [0.1, 0.05],
    'max_depth': [5, 6, 7, 8],
    'min_child_weight': [1,2,4,8,16,32]
    }

    params = []
    param_init = {
    'colsample_bytree': 1.0,
    'colsample_bylevel': 0.3,
    'subsample': 0.9,
    'gamma': 0,
    'lambda': 1,
    'alpha': 0,
    'objective': 'reg:linear',
    'silent': 1,
    'random_state': 71,
    'evalmetric': 'rmse',
    }
    
    trial = 15 # the num of trial
    score = 10000 # should be bigger value
    
    #start trial
    for i in range(trial):
        param = param_init
        
        for k, v in parameters_tobe_selected.items():
            param[k] = np.random.choice(v)
        
        #store paremeters for each trial
        params.append(param)
        
        #making model
        model = xgb.train(param, dtrain, num_round, evals=watchlist)
        
        #validation score
        y_pred_vali = model.predict(dvalid)
        valid_score = mean_squared_error(y_valid, y_pred_vali)
        if valid_score < score:
            score = valid_score
            final_param = param
            #print(f'mse: {math.sqrt(score):.4f}')
    
    #print(f'mse: {math.sqrt(score):.4f}')
    #print(final_param)
    
    return xgb.train(final_param, dtrain, num_round, evals=watchlist), score, final_param

In [504]:
def build_xgb_model2(x_train, y_train):
    """
    Build Model function
    
    This function output is a Scikit ML Pipeline that process text messages
    according to NLP best-practice and apply a classifier.
    return a trained model and score
    """
    
    
    #setting parameters
    parameters_tobe_selected = {
    'eta': [0.1, 0.05],
    'max_depth': [5, 6, 7, 8],
    'min_child_weight': [1,2,4,8,16,32]
    }

    params = []
    param_init = {
    'colsample_bytree': 1.0,
    'colsample_bylevel': 0.3,
    'subsample': 0.9,
    'gamma': 0,
    'lambda': 1,
    'alpha': 0,
    'objective': 'reg:linear',
    'silent': 1,
    'random_state': 71,
    'evalmetric': 'rmse',
    }
    
    trial = 20 # the num of trial
    great_score = 10000.0 # should be bigger value
    
    #start trial
    for i in range(trial):
        param = param_init
        
        #setting parameters for each trial
        #random choice for to-be-selected patameters
        for k, v in parameters_tobe_selected.items():
            param[k] = np.random.choice(v)
        
        #store paremeters for each trial
        #params.append(param)
        
        # for each trial excute k-fold validation
        kf = KFold(n_splits=5, shuffle=True, random_state=71)
        valid_scores = [] # to calculate mean for this trial
        for tr_idx, va_idx in kf.split(x_train):
            
            tr_x, va_x = x_train.iloc[tr_idx], x_train.iloc[va_idx]
            tr_y, va_y = y_train.iloc[tr_idx], y_train.iloc[va_idx]
            
            #change the datatype to fit to xgb
            dtrain = xgb.DMatrix(tr_x, label = tr_y)
            dvalid = xgb.DMatrix(va_x, label = va_y)

            #making model
            watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
            num_round = 50        

            print('Building model...')
            #making
            model = xgb.train(param, dtrain, num_round, evals=watchlist)       

            #validation score
            y_pred_vali = model.predict(dvalid)
            valid_score = mean_squared_error(va_y, y_pred_vali)
            
            valid_scores.append(valid_score)
        
        mean_for_this_validation = np.mean(valid_scores)    
        if great_score > mean_for_this_validation:
            great_score = mean_for_this_validation
            final_param = param
         
    #print(f'mse: {math.sqrt(score):.4f}')
    #print(final_param)
    
    return xgb.train(final_param, dtrain, num_round, evals=watchlist), great_score, final_param

In [489]:
def evaluate_model(model, X_test, Y_test, Y_test_IDs):
    """
    Evaluate Model function
    
    This function applies ML pipeline to a test set and prints out
    model performance (mse)
    
    Arguments:
        model -> Scikit ML Pipeline
        X_test -> test features
        Y_test -> test true price
    
    Output: result dataframe contains ID, MSE, Y_test, Y_pred
    """
    Y_pred = model.predict(X_test)
    mse = mean_squared_error(Y_test, Y_pred)
    
    results = [Y_test_IDs, mse, Y_test, Y_pred]
    results_Series = [pd.Series(r) for r in results]
    df_result = pd.concat(results_Series, names = ['ID', 'MSE', 'Y_test','Y_pred'])
    return df_result

In [490]:
def save_model(model, model_filepath):
    """
    Save Model function
    
    This function saves trained model as Pickle file, to be loaded later.
    
    Arguments:
        model -> GridSearchCV or Scikit Pipelin object
        model_filepath -> destination path to save .pkl file
    
    """    
    with open(model_filepath, 'wb') as file:
        pickle.dump(model, file)

In [491]:
#main

In [492]:
IDs_train = df_train["Id"]
IDs_test = df_test["Id"]

In [493]:
#normalization for sales price
Y_train = df_train["SalePrice"]
Y_train_min = Y_train.min()
Y_train_max = Y_train.max()
Y_train = (Y_train - Y_train_min) / (Y_train_max - Y_train_min)
Y_train

0       0.241078
1       0.203583
2       0.261908
3       0.145952
4       0.298709
          ...   
1455    0.194556
1456    0.243161
1457    0.321622
1458    0.148903
1459    0.156367
Name: SalePrice, Length: 1460, dtype: float64

In [494]:
X_train = df_train.drop(["Id", "SalePrice"], axis=1)
#Y_test = df_test["SalePrice"]
X_test = df_test.drop(["Id"], axis=1)#, "SalePrice"])

In [495]:
X_train.shape

(1460, 79)

In [496]:
X_test.shape

(1459, 79)

In [497]:
X_train, X_test = data_preprocessing(X_train, X_test)

In [498]:
X_train_encorded, X_test_encorded = Label_encoding(X_train, X_test)

Street
Alley
LotShape
LandContour
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
OverallQual
OverallCond
RoofStyle
RoofMatl
MasVnrType
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Heating
HeatingQC
CentralAir
Electrical
BedroomAbvGr
KitchenAbvGr
FireplaceQu
GarageType
GarageFinish
GarageQual
GarageCond
PavedDrive
PoolQC
Fence
MiscFeature
MoSold
SaleCondition


In [505]:
#building model
print('Building model...')
model, score, param = build_xgb_model2(X_train_encorded, Y_train)

Building model...
Building model...
[0]	train-rmse:0.301571	eval-rmse:0.303104
[1]	train-rmse:0.28714	eval-rmse:0.288742
[2]	train-rmse:0.273355	eval-rmse:0.27488
[3]	train-rmse:0.260346	eval-rmse:0.261911
[4]	train-rmse:0.248003	eval-rmse:0.249402
[5]	train-rmse:0.23628	eval-rmse:0.237556
[6]	train-rmse:0.225113	eval-rmse:0.226414
[7]	train-rmse:0.214506	eval-rmse:0.21587
[8]	train-rmse:0.204389	eval-rmse:0.20579
[9]	train-rmse:0.194811	eval-rmse:0.196281
[10]	train-rmse:0.185721	eval-rmse:0.187166
[11]	train-rmse:0.177151	eval-rmse:0.178657
[12]	train-rmse:0.16891	eval-rmse:0.170441
[13]	train-rmse:0.161113	eval-rmse:0.162609
[14]	train-rmse:0.153772	eval-rmse:0.155375
[15]	train-rmse:0.146788	eval-rmse:0.148535
[16]	train-rmse:0.140113	eval-rmse:0.141949
[17]	train-rmse:0.133833	eval-rmse:0.135695
[18]	train-rmse:0.127825	eval-rmse:0.129946
[19]	train-rmse:0.122141	eval-rmse:0.124431
[20]	train-rmse:0.116727	eval-rmse:0.119325
[21]	train-rmse:0.111646	eval-rmse:0.114278
[22]	train-r

/Users/yumasasaka/anaconda3/envs/Kaggle/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[30]	train-rmse:0.076114	eval-rmse:0.080973
[31]	train-rmse:0.073206	eval-rmse:0.078367
[32]	train-rmse:0.070466	eval-rmse:0.075813
[33]	train-rmse:0.067814	eval-rmse:0.073495
[34]	train-rmse:0.065342	eval-rmse:0.071203
[35]	train-rmse:0.063036	eval-rmse:0.069162
[36]	train-rmse:0.060803	eval-rmse:0.067244
[37]	train-rmse:0.058689	eval-rmse:0.065442
[38]	train-rmse:0.056714	eval-rmse:0.063739
[39]	train-rmse:0.054884	eval-rmse:0.062088
[40]	train-rmse:0.053156	eval-rmse:0.060566
[41]	train-rmse:0.051544	eval-rmse:0.059211
[42]	train-rmse:0.050025	eval-rmse:0.057903
[43]	train-rmse:0.048607	eval-rmse:0.056646
[44]	train-rmse:0.047224	eval-rmse:0.055519
[45]	train-rmse:0.045915	eval-rmse:0.054558
[46]	train-rmse:0.044747	eval-rmse:0.05372
[47]	train-rmse:0.043611	eval-rmse:0.052879
[48]	train-rmse:0.042519	eval-rmse:0.052234
[49]	train-rmse:0.041535	eval-rmse:0.051641
Building model...
[0]	train-rmse:0.302575	eval-rmse:0.298536
[1]	train-rmse:0.288132	eval-rmse:0.284088
[2]	train-rmse:0.

/Users/yumasasaka/anaconda3/envs/Kaggle/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[36]	train-rmse:0.061435	eval-rmse:0.062006
[37]	train-rmse:0.059365	eval-rmse:0.060059
[38]	train-rmse:0.05743	eval-rmse:0.058257
[39]	train-rmse:0.055631	eval-rmse:0.056547
[40]	train-rmse:0.053867	eval-rmse:0.054973
[41]	train-rmse:0.052261	eval-rmse:0.053507
[42]	train-rmse:0.050757	eval-rmse:0.052063
[43]	train-rmse:0.049249	eval-rmse:0.050773
[44]	train-rmse:0.047924	eval-rmse:0.049576
[45]	train-rmse:0.046665	eval-rmse:0.04842
[46]	train-rmse:0.045481	eval-rmse:0.047306
[47]	train-rmse:0.04438	eval-rmse:0.046304
[48]	train-rmse:0.043355	eval-rmse:0.045348
[49]	train-rmse:0.042429	eval-rmse:0.044498
Building model...
[0]	train-rmse:0.301857	eval-rmse:0.30171
[1]	train-rmse:0.287382	eval-rmse:0.287465
[2]	train-rmse:0.273639	eval-rmse:0.273861
[3]	train-rmse:0.260607	eval-rmse:0.261074
[4]	train-rmse:0.248234	eval-rmse:0.24881
[5]	train-rmse:0.236487	eval-rmse:0.237234
[6]	train-rmse:0.225236	eval-rmse:0.226071
[7]	train-rmse:0.214649	eval-rmse:0.21563
[8]	train-rmse:0.20453	eval-

...

[40]	train-rmse:0.05273	eval-rmse:0.061062
[41]	train-rmse:0.051106	eval-rmse:0.059666
[42]	train-rmse:0.049562	eval-rmse:0.058456
[43]	train-rmse:0.04809	eval-rmse:0.057275
[44]	train-rmse:0.046715	eval-rmse:0.056141
[45]	train-rmse:0.045401	eval-rmse:0.055105
[46]	train-rmse:0.044162	eval-rmse:0.054095
[47]	train-rmse:0.043023	eval-rmse:0.05325
[48]	train-rmse:0.041976	eval-rmse:0.052402
[49]	train-rmse:0.041	eval-rmse:0.051593
Building model...
[0]	train-rmse:0.301215	eval-rmse:0.304419
[1]	train-rmse:0.286779	eval-rmse:0.290113
[2]	train-rmse:0.273033	eval-rmse:0.276582
[3]	train-rmse:0.260028	eval-rmse:0.263769
[4]	train-rmse:0.24765	eval-rmse:0.251641
[5]	train-rmse:0.235958	eval-rmse:0.240361
[6]	train-rmse:0.224826	eval-rmse:0.229573
[7]	train-rmse:0.214239	eval-rmse:0.219169
[8]	train-rmse:0.204184	eval-rmse:0.209324
[9]	train-rmse:0.194615	eval-rmse:0.199862
[10]	train-rmse:0.185546	eval-rmse:0.190971
[11]	train-rmse:0.176942	eval-rmse:0.182613
[12]	train-rmse:0.1687	eval-rms

/Users/yumasasaka/anaconda3/envs/Kaggle/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[41]	train-rmse:0.051006	eval-rmse:0.064119
[42]	train-rmse:0.049424	eval-rmse:0.062789
[43]	train-rmse:0.047956	eval-rmse:0.061511
[44]	train-rmse:0.046542	eval-rmse:0.060431
[45]	train-rmse:0.045269	eval-rmse:0.059293
[46]	train-rmse:0.044042	eval-rmse:0.058259
[47]	train-rmse:0.042917	eval-rmse:0.057278
[48]	train-rmse:0.041815	eval-rmse:0.056406
[49]	train-rmse:0.040744	eval-rmse:0.055559
Building model...
[0]	train-rmse:0.301879	eval-rmse:0.301709
[1]	train-rmse:0.28748	eval-rmse:0.287308
[2]	train-rmse:0.273703	eval-rmse:0.273447
[3]	train-rmse:0.260655	eval-rmse:0.260243
[4]	train-rmse:0.248278	eval-rmse:0.247974
[5]	train-rmse:0.23661	eval-rmse:0.236339
[6]	train-rmse:0.225417	eval-rmse:0.225205
[7]	train-rmse:0.214843	eval-rmse:0.21455
[8]	train-rmse:0.2048	eval-rmse:0.204585
[9]	train-rmse:0.195223	eval-rmse:0.19498
[10]	train-rmse:0.186173	eval-rmse:0.185931
[11]	train-rmse:0.177602	eval-rmse:0.177428
[12]	train-rmse:0.169377	eval-rmse:0.169142
[13]	train-rmse:0.161625	eval-

/Users/yumasasaka/anaconda3/envs/Kaggle/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[40]	train-rmse:0.054176	eval-rmse:0.054655
[41]	train-rmse:0.052553	eval-rmse:0.053133
[42]	train-rmse:0.051028	eval-rmse:0.051723
[43]	train-rmse:0.049583	eval-rmse:0.050506
[44]	train-rmse:0.048224	eval-rmse:0.049228
[45]	train-rmse:0.046959	eval-rmse:0.04801
[46]	train-rmse:0.045699	eval-rmse:0.04709
[47]	train-rmse:0.044595	eval-rmse:0.046075
[48]	train-rmse:0.04354	eval-rmse:0.045176
[49]	train-rmse:0.042539	eval-rmse:0.044297
[0]	train-rmse:0.301879	eval-rmse:0.301709
[1]	train-rmse:0.28748	eval-rmse:0.287308
[2]	train-rmse:0.273703	eval-rmse:0.273447
[3]	train-rmse:0.260655	eval-rmse:0.260243
[4]	train-rmse:0.248278	eval-rmse:0.247974
[5]	train-rmse:0.23661	eval-rmse:0.236339
[6]	train-rmse:0.225417	eval-rmse:0.225205
[7]	train-rmse:0.214843	eval-rmse:0.21455
[8]	train-rmse:0.2048	eval-rmse:0.204585
[9]	train-rmse:0.195223	eval-rmse:0.19498
[10]	train-rmse:0.186173	eval-rmse:0.185931
[11]	train-rmse:0.177602	eval-rmse:0.177428
[12]	train-rmse:0.169377	eval-rmse:0.169142
[13]	tr

In [506]:
score

0.0015583802732897188

In [507]:
param

{'colsample_bytree': 1.0,
 'colsample_bylevel': 0.3,
 'subsample': 0.9,
 'gamma': 0,
 'lambda': 1,
 'alpha': 0,
 'objective': 'reg:linear',
 'silent': 1,
 'random_state': 71,
 'evalmetric': 'rmse',
 'eta': 0.05,
 'max_depth': 8,
 'min_child_weight': 16}

In [510]:
dtest = xgb.DMatrix(X_test)
Y_pred_test = model.predict(dtest)

In [516]:
Y_pred_test[0]

0.15234998

In [534]:
#concat ID and the result
final_result = pd.concat([IDs_test, pd.DataFrame(Y_pred_test, columns=["SalePrice"])], axis=1)

#transform back the result to estimated sales price
final_result["SalePrice"] = final_result["SalePrice"].apply(lambda x : x * (Y_train_max - Y_train_min) + Y_train_min)

In [535]:
final_result

,Id,SalePrice
0,1461,144607.219651
1,1462,179680.320868
2,1463,196198.756668
3,1464,198001.644611
4,1465,205929.016444
...,...,...
1454,2915,116470.051050
1455,2916,117398.202804
1456,2917,183746.749657
1457,2918,144266.574717


In [536]:
final_result.to_csv("submission1.csv", index=False)

In [471]:
#X_train_encorded, X_valid_encorded, Y_train, Y_valid = train_test_split(X_train_encorded, Y_train, test_size=0.2)
kf = KFold(n_splits=5, shuffle=True, random_state=71)
scores = []
params = []
for tr_idx, va_idx in kf.split(X_train_encorded):
    tr_x, va_x = X_train_encorded.iloc[tr_idx], X_train_encorded.iloc[va_idx]
    tr_y, va_y = Y_train.iloc[tr_idx], Y_train.iloc[va_idx]
    
    print('Building model...')
    model, score, param = build_xgb_model(tr_x, va_x, tr_y, va_y)
    scores.append(score)
    params.append(param)

Building model...
[0]	train-rmse:0.301426	eval-rmse:0.303016
[1]	train-rmse:0.286868	eval-rmse:0.28863
[2]	train-rmse:0.272973	eval-rmse:0.274779
[3]	train-rmse:0.259845	eval-rmse:0.261782
[4]	train-rmse:0.247352	eval-rmse:0.249382
[5]	train-rmse:0.235452	eval-rmse:0.237422
[6]	train-rmse:0.224137	eval-rmse:0.226249
[7]	train-rmse:0.213372	eval-rmse:0.215668
[8]	train-rmse:0.203131	eval-rmse:0.205476
[9]	train-rmse:0.193432	eval-rmse:0.195912
[10]	train-rmse:0.18418	eval-rmse:0.186886
[11]	train-rmse:0.175452	eval-rmse:0.178336
[12]	train-rmse:0.167115	eval-rmse:0.170177
[13]	train-rmse:0.15917	eval-rmse:0.162285
[14]	train-rmse:0.151683	eval-rmse:0.155035
[15]	train-rmse:0.144538	eval-rmse:0.148193
[16]	train-rmse:0.13768	eval-rmse:0.141624
[17]	train-rmse:0.13121	eval-rmse:0.135401
[18]	train-rmse:0.12508	eval-rmse:0.129407
[19]	train-rmse:0.119267	eval-rmse:0.123839
[20]	train-rmse:0.113713	eval-rmse:0.118537
[21]	train-rmse:0.108443	eval-rmse:0.113514
[22]	train-rmse:0.103419	eval-

/Users/yumasasaka/anaconda3/envs/Kaggle/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[25]	train-rmse:0.089839	eval-rmse:0.096322
[26]	train-rmse:0.085712	eval-rmse:0.092539
[27]	train-rmse:0.081851	eval-rmse:0.089001
[28]	train-rmse:0.07817	eval-rmse:0.085866
[29]	train-rmse:0.074655	eval-rmse:0.082826
[30]	train-rmse:0.071302	eval-rmse:0.079955
[31]	train-rmse:0.068154	eval-rmse:0.077333
[32]	train-rmse:0.065177	eval-rmse:0.074678
[33]	train-rmse:0.062328	eval-rmse:0.072356
[34]	train-rmse:0.059632	eval-rmse:0.070156
[35]	train-rmse:0.057063	eval-rmse:0.06819
[36]	train-rmse:0.054628	eval-rmse:0.066304
[37]	train-rmse:0.0523	eval-rmse:0.06446
[38]	train-rmse:0.050077	eval-rmse:0.062669
[39]	train-rmse:0.048007	eval-rmse:0.06116
[40]	train-rmse:0.04603	eval-rmse:0.059763
[41]	train-rmse:0.044155	eval-rmse:0.058447
[42]	train-rmse:0.042361	eval-rmse:0.057131
[43]	train-rmse:0.040652	eval-rmse:0.055853
[44]	train-rmse:0.039022	eval-rmse:0.054934
[45]	train-rmse:0.037479	eval-rmse:0.053967
[46]	train-rmse:0.036004	eval-rmse:0.05315
[47]	train-rmse:0.034613	eval-rmse:0.052

[14]	train-rmse:0.152982	eval-rmse:0.155372
[15]	train-rmse:0.145938	eval-rmse:0.148539
[16]	train-rmse:0.139205	eval-rmse:0.14203
[17]	train-rmse:0.132865	eval-rmse:0.135821
[18]	train-rmse:0.126799	eval-rmse:0.130195
[19]	train-rmse:0.121064	eval-rmse:0.124641
[20]	train-rmse:0.115609	eval-rmse:0.119467
[21]	train-rmse:0.110443	eval-rmse:0.114503
[22]	train-rmse:0.105494	eval-rmse:0.109736
[23]	train-rmse:0.100814	eval-rmse:0.10531
[24]	train-rmse:0.096377	eval-rmse:0.101284
[25]	train-rmse:0.092165	eval-rmse:0.097549
[26]	train-rmse:0.088158	eval-rmse:0.093921
[27]	train-rmse:0.084412	eval-rmse:0.090485
[28]	train-rmse:0.080839	eval-rmse:0.087388
[29]	train-rmse:0.077436	eval-rmse:0.084431
[30]	train-rmse:0.074223	eval-rmse:0.081597
[31]	train-rmse:0.071247	eval-rmse:0.07894
[32]	train-rmse:0.068419	eval-rmse:0.076386
[33]	train-rmse:0.06568	eval-rmse:0.074201
[34]	train-rmse:0.063154	eval-rmse:0.072106
[35]	train-rmse:0.060789	eval-rmse:0.070142
[36]	train-rmse:0.058466	eval-rmse:0

[3]	train-rmse:0.259813	eval-rmse:0.261855
[4]	train-rmse:0.247318	eval-rmse:0.249478
[5]	train-rmse:0.235413	eval-rmse:0.23753
[6]	train-rmse:0.224098	eval-rmse:0.226362
[7]	train-rmse:0.213335	eval-rmse:0.215814
[8]	train-rmse:0.203087	eval-rmse:0.205629
[9]	train-rmse:0.193388	eval-rmse:0.196092
[10]	train-rmse:0.184128	eval-rmse:0.187042
[11]	train-rmse:0.175392	eval-rmse:0.178483
[12]	train-rmse:0.167046	eval-rmse:0.170355
[13]	train-rmse:0.159107	eval-rmse:0.162442
[14]	train-rmse:0.15162	eval-rmse:0.155206
[15]	train-rmse:0.144485	eval-rmse:0.14839
[16]	train-rmse:0.137617	eval-rmse:0.141833
[17]	train-rmse:0.131143	eval-rmse:0.135646
[18]	train-rmse:0.125	eval-rmse:0.129714
[19]	train-rmse:0.119175	eval-rmse:0.124197
[20]	train-rmse:0.113621	eval-rmse:0.1189
[21]	train-rmse:0.108346	eval-rmse:0.113878
[22]	train-rmse:0.103319	eval-rmse:0.109064
[23]	train-rmse:0.098509	eval-rmse:0.104599
[24]	train-rmse:0.094001	eval-rmse:0.100533
[25]	train-rmse:0.08971	eval-rmse:0.096654
[26]

...

[27]	train-rmse:0.02787	eval-rmse:0.037142
[28]	train-rmse:0.026274	eval-rmse:0.036107
[29]	train-rmse:0.024928	eval-rmse:0.035242
[30]	train-rmse:0.02375	eval-rmse:0.034264
[31]	train-rmse:0.022755	eval-rmse:0.033709
[32]	train-rmse:0.021862	eval-rmse:0.033082
[33]	train-rmse:0.020996	eval-rmse:0.032689
[34]	train-rmse:0.020241	eval-rmse:0.032347
[35]	train-rmse:0.019576	eval-rmse:0.032099
[36]	train-rmse:0.01893	eval-rmse:0.032
[37]	train-rmse:0.018489	eval-rmse:0.031742
[38]	train-rmse:0.017958	eval-rmse:0.031485
[39]	train-rmse:0.017558	eval-rmse:0.031235
[40]	train-rmse:0.017217	eval-rmse:0.031013
[41]	train-rmse:0.016879	eval-rmse:0.030897
[42]	train-rmse:0.016622	eval-rmse:0.030723
[43]	train-rmse:0.016328	eval-rmse:0.030663
[44]	train-rmse:0.016077	eval-rmse:0.03047
[45]	train-rmse:0.015838	eval-rmse:0.03045
[46]	train-rmse:0.015553	eval-rmse:0.030336
[47]	train-rmse:0.015305	eval-rmse:0.030258
[48]	train-rmse:0.015071	eval-rmse:0.030177
[49]	train-rmse:0.014861	eval-rmse:0.030

[16]	train-rmse:0.061245	eval-rmse:0.066355
[17]	train-rmse:0.056103	eval-rmse:0.06172
[18]	train-rmse:0.051434	eval-rmse:0.05753
[19]	train-rmse:0.047407	eval-rmse:0.054031
[20]	train-rmse:0.043777	eval-rmse:0.05093
[21]	train-rmse:0.040594	eval-rmse:0.048386
[22]	train-rmse:0.037709	eval-rmse:0.046015
[23]	train-rmse:0.03506	eval-rmse:0.044044
[24]	train-rmse:0.032807	eval-rmse:0.042487
[25]	train-rmse:0.030633	eval-rmse:0.041002
[26]	train-rmse:0.028818	eval-rmse:0.039659
[27]	train-rmse:0.0272	eval-rmse:0.038775
[28]	train-rmse:0.025843	eval-rmse:0.037859
[29]	train-rmse:0.024566	eval-rmse:0.037053
[30]	train-rmse:0.023465	eval-rmse:0.036385
[31]	train-rmse:0.022432	eval-rmse:0.03569
[32]	train-rmse:0.02152	eval-rmse:0.0351
[33]	train-rmse:0.020751	eval-rmse:0.034778
[34]	train-rmse:0.020058	eval-rmse:0.034412
[35]	train-rmse:0.019359	eval-rmse:0.034135
[36]	train-rmse:0.018669	eval-rmse:0.033829
[37]	train-rmse:0.018184	eval-rmse:0.033736
[38]	train-rmse:0.017672	eval-rmse:0.03356

[4]	train-rmse:0.247536	eval-rmse:0.248027
[5]	train-rmse:0.235594	eval-rmse:0.236206
[6]	train-rmse:0.224246	eval-rmse:0.224975
[7]	train-rmse:0.213458	eval-rmse:0.214202
[8]	train-rmse:0.203273	eval-rmse:0.204175
[9]	train-rmse:0.193566	eval-rmse:0.194554
[10]	train-rmse:0.184331	eval-rmse:0.185517
[11]	train-rmse:0.175537	eval-rmse:0.177017
[12]	train-rmse:0.16721	eval-rmse:0.168908
[13]	train-rmse:0.159303	eval-rmse:0.161148
[14]	train-rmse:0.151782	eval-rmse:0.153876
[15]	train-rmse:0.14464	eval-rmse:0.146918
[16]	train-rmse:0.137833	eval-rmse:0.140233
[17]	train-rmse:0.13131	eval-rmse:0.133809
[18]	train-rmse:0.12514	eval-rmse:0.127786
[19]	train-rmse:0.119288	eval-rmse:0.122093
[20]	train-rmse:0.113712	eval-rmse:0.116676
[21]	train-rmse:0.108461	eval-rmse:0.111629
[22]	train-rmse:0.103471	eval-rmse:0.106839
[23]	train-rmse:0.098681	eval-rmse:0.10222
[24]	train-rmse:0.094196	eval-rmse:0.098035
[25]	train-rmse:0.089887	eval-rmse:0.093997
[26]	train-rmse:0.085816	eval-rmse:0.090142

[43]	train-rmse:0.048021	eval-rmse:0.050333
[44]	train-rmse:0.046565	eval-rmse:0.048832
[45]	train-rmse:0.045301	eval-rmse:0.047667
[46]	train-rmse:0.044016	eval-rmse:0.04646
[47]	train-rmse:0.042892	eval-rmse:0.045433
[48]	train-rmse:0.041788	eval-rmse:0.044467
[49]	train-rmse:0.040683	eval-rmse:0.043493
[0]	train-rmse:0.30175	eval-rmse:0.301675
[1]	train-rmse:0.287219	eval-rmse:0.287152
[2]	train-rmse:0.273306	eval-rmse:0.273229
[3]	train-rmse:0.260128	eval-rmse:0.260033
[4]	train-rmse:0.247641	eval-rmse:0.24776
[5]	train-rmse:0.235712	eval-rmse:0.235932
[6]	train-rmse:0.224447	eval-rmse:0.224733
[7]	train-rmse:0.213664	eval-rmse:0.213996
[8]	train-rmse:0.203477	eval-rmse:0.20393
[9]	train-rmse:0.193793	eval-rmse:0.194357
[10]	train-rmse:0.184561	eval-rmse:0.185375
[11]	train-rmse:0.175777	eval-rmse:0.176729
[12]	train-rmse:0.167479	eval-rmse:0.168512
[13]	train-rmse:0.159575	eval-rmse:0.160631
[14]	train-rmse:0.152039	eval-rmse:0.153128
[15]	train-rmse:0.144888	eval-rmse:0.146105
[1

[32]	train-rmse:0.038939	eval-rmse:0.038572
[33]	train-rmse:0.038505	eval-rmse:0.038246
[34]	train-rmse:0.038081	eval-rmse:0.037795
[35]	train-rmse:0.037556	eval-rmse:0.037597
[36]	train-rmse:0.037192	eval-rmse:0.037244
[37]	train-rmse:0.036778	eval-rmse:0.037237
[38]	train-rmse:0.036373	eval-rmse:0.037046
[39]	train-rmse:0.036058	eval-rmse:0.03676
[40]	train-rmse:0.035799	eval-rmse:0.036605
[41]	train-rmse:0.035612	eval-rmse:0.036442
[42]	train-rmse:0.035411	eval-rmse:0.036302
[43]	train-rmse:0.035217	eval-rmse:0.036178
[44]	train-rmse:0.035056	eval-rmse:0.03596
[45]	train-rmse:0.034809	eval-rmse:0.035904
[46]	train-rmse:0.034615	eval-rmse:0.035988
[47]	train-rmse:0.034469	eval-rmse:0.035916
[48]	train-rmse:0.034217	eval-rmse:0.035639
[49]	train-rmse:0.034044	eval-rmse:0.035604


In [472]:
print(np.mean(scores))

0.0014084022655457683


In [479]:
for i in range(len(params)):
    print("------------------------------------")
    print("score: " + str(scores[i]))
    print("------------------------------------")
    print(params[i])
    print("------------------------------------")
    print("------------------------------------")

------------------------------------
score: 0.0017561957225798739
------------------------------------
{'colsample_bytree': 1.0, 'colsample_bylevel': 0.3, 'subsample': 0.9, 'gamma': 0, 'lambda': 1, 'alpha': 0, 'objective': 'reg:linear', 'silent': 1, 'random_state': 71, 'evalmetric': 'rmse', 'eta': 0.05, 'max_depth': 8, 'min_child_weight': 8}
------------------------------------
------------------------------------
------------------------------------
score: 0.0011234049555207339
------------------------------------
{'colsample_bytree': 1.0, 'colsample_bylevel': 0.3, 'subsample': 0.9, 'gamma': 0, 'lambda': 1, 'alpha': 0, 'objective': 'reg:linear', 'silent': 1, 'random_state': 71, 'evalmetric': 'rmse', 'eta': 0.1, 'max_depth': 7, 'min_child_weight': 16}
------------------------------------
------------------------------------
------------------------------------
score: 0.0014824678996487005
------------------------------------
{'colsample_bytree': 1.0, 'colsample_bylevel': 0.3, 'subsampl

In [466]:
#print('Building model...')
#model = build_xgb_model(X_train_encorded, X_valid_encorded, Y_train, Y_valid)